In [1]:
# backtest d'une strategie de trading utilisant les moyennes mobiles comme indicateurs technique

In [4]:
pip install python-binance

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ta

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [57]:
# Import des libraries necessaire

import pandas as pd
from binance.client import Client
import ta

In [58]:
# Importer la dataFrame
historicalData = Client().get_historical_klines("BTCUSDT",Client.KLINE_INTERVAL_1HOUR,"01 January 2017")
data = pd.DataFrame(historicalData, columns = ['timestamp', 'open','high','low','close','volume','close_time','quote_av','trades','tb_base_av','tb_quote_av','ignore'])

In [60]:
# Selectionner les donnees à utiliser 
data['close'] = pd.to_numeric(data['close'])
data['SMA200'] = ta.trend.sma_indicator(data['close'], 200)
data['SMA600'] = ta.trend.sma_indicator(data['close'], 600)

# Utiliser le Timestamp comme index
data_used = data[['close','SMA200', 'SMA600']].set_index(data['timestamp'])
data_used.index = pd.to_datetime(data_used.index, unit = 'ms')
data_used.head()

,close,SMA200,SMA600
timestamp,,,
2017-08-17 04:00:00,4308.83,NaN,NaN
2017-08-17 05:00:00,4315.32,NaN,NaN
2017-08-17 06:00:00,4324.35,NaN,NaN
2017-08-17 07:00:00,4349.99,NaN,NaN
2017-08-17 08:00:00,4360.69,NaN,NaN


In [62]:
# Commencer a 1000 USDT et 0 BTC

usdt = 1000
btc = 0
lastIndex = data_used.first_valid_index()

for index, row in data_used.iterrows() :
    # Acheter quand la SMA_200 est superieur à la SMA_600
    if data_used['SMA200'][lastIndex] > data_used['SMA600'][lastIndex] and usdt > 10 :
        btc = usdt / data_used['close'][index]
        btc = btc - 0.0007 * btc  
        usdt = 0
        
        print("Acheter le bitcoin a", data_used['close'][index],'$ Le', index)

    # Vendre quand la SMA_200 est inferieur à la SMA_600    
    if data_used['SMA200'][lastIndex] < data_used['SMA600'][lastIndex] and btc > 0.0001 :
        usdt = btc * data_used['close'][index]
        usdt = usdt - 0.0007 * usdt 
        btc = 0
        
        print("Vendre le bitcoin a", data_used['close'][index],'$ Le', index)
    
    lastIndex = index

Acheter le bitcoin a 4176.24 $ Le 2017-09-11 10:00:00
Vendre le bitcoin a 4259.44 $ Le 2017-09-12 13:00:00
Acheter le bitcoin a 4302.02 $ Le 2017-10-01 08:00:00
Vendre le bitcoin a 16351.02 $ Le 2017-12-27 05:00:00
Acheter le bitcoin a 14219.61 $ Le 2018-01-10 06:00:00
Vendre le bitcoin a 14195.0 $ Le 2018-01-15 12:00:00
Acheter le bitcoin a 11082.94 $ Le 2018-02-19 18:00:00
Vendre le bitcoin a 8704.0 $ Le 2018-03-11 07:00:00
Acheter le bitcoin a 8018.98 $ Le 2018-04-16 21:00:00
Vendre le bitcoin a 8665.97 $ Le 2018-05-14 00:00:00
Acheter le bitcoin a 6584.36 $ Le 2018-07-07 10:00:00
Vendre le bitcoin a 7087.99 $ Le 2018-08-07 11:00:00
Acheter le bitcoin a 6902.74 $ Le 2018-08-28 04:00:00
Vendre le bitcoin a 6355.0 $ Le 2018-09-11 03:00:00
Acheter le bitcoin a 6669.36 $ Le 2018-09-27 19:00:00
Vendre le bitcoin a 6309.02 $ Le 2018-10-12 22:00:00
Acheter le bitcoin a 6573.58 $ Le 2018-10-20 10:00:00
Vendre le bitcoin a 6491.85 $ Le 2018-10-28 10:00:00
Acheter le bitcoin a 4125.4 $ Le 201

In [63]:
finalResult = usdt + btc * data_used['close'].iloc[-1] 
print("Resultat Finale",finalResult,'USDT')

Resultat Finale 17302.470145622377 USDT


In [64]:
print("acheter a la premiere date et vendre a la derniere date", (1000 / data_used['close'].iloc[0]) * data_used['close'].iloc[-1],'USDT')

acheter a la premiere date et vendre a la derniere date 3836.677241849876 USDT
